# Review Spacy No.2

Example of training spaCy's named entity recognizer, starting off with an
existing model or a blank model.

In [4]:
%run -i 'script_examples/train_ner.py'

Created blank 'en' model
Losses {'ner': 9.899998903274536}
Losses {'ner': 9.654258012771606}
Losses {'ner': 9.517346858978271}
Losses {'ner': 9.254633784294128}
Losses {'ner': 9.026497721672058}
Losses {'ner': 8.36333966255188}
Losses {'ner': 8.172319173812866}
Losses {'ner': 7.955441236495972}
Losses {'ner': 7.785399794578552}
Losses {'ner': 7.004211902618408}
Losses {'ner': 6.142524182796478}
Losses {'ner': 6.061682462692261}
Losses {'ner': 5.565993428230286}
Losses {'ner': 5.502236217260361}
Losses {'ner': 5.09660404920578}
Losses {'ner': 5.113696753978729}
Losses {'ner': 4.525115683674812}
Losses {'ner': 4.533860921859741}
Losses {'ner': 4.947815239429474}
Losses {'ner': 4.473199754953384}
Losses {'ner': 4.68527789786458}
Losses {'ner': 4.513813484460115}
Losses {'ner': 3.8508487790822983}
Losses {'ner': 4.436941131949425}
Losses {'ner': 4.509779965039343}
Losses {'ner': 3.513655934482813}
Losses {'ner': 4.269340425729752}
Losses {'ner': 3.432205328717828}
Losses {'ner': 2.97814990

## Revie Model 2

In [5]:
import spacy

In [6]:
nlp=spacy.load("en_core_web_sm")

In [7]:
text=open("data/GM_News.txt").read()

In [8]:
print(text)

GM said Monday it will invest $2.2 billion into its Detroit-Hamtramck assembly plant to produce all-electric trucks and SUVs as well as a self-driving vehicle unveiled by its subsidiary Cruise. The automaker will invest an additional $800 million in supplier tooling and other projects related to the launch of the new electric trucks.

GM will kick off this new program with an all-electric pickup truck that will go into production in late 2021. The Cruise Origin, the electric self-driving shuttle designed for ride sharing, will be the second vehicle to go into production at the Detroit area plant.

Detroit-Hamtramck will be GM’s first fully-dedicated electric vehicle assembly plant. When fully operational, the plant will create more than 2,200 jobs, according to GM.


In [11]:
#Review NER Detection
from spacy import displacy

doc = nlp(text)
displacy.render(doc, style="ent")

Now I try to detect the words **{'SUV','Cruise'}** as entities.

The Pipeline:
1.- Train Data definition
2.- Main funtion to training the model
3.-

In [ ]:
unveiled by its subsidiary Cruise

In [28]:
# training data
TRAIN_DATA = [
    ("all-electric trucks and SUVs as well as a self-driving vehicle", {"entities": [(25,28, "FAC")]}),
    ("unveiled by its subsidiary Cruise", {"entities": [(28, 34, "PRODUCT")]}),
    ("The Cruise Origin is powered by a brand-new",{"entities": [(4, 10, "PRODUCT")]}),
    ("Find out which SUVs are #1 in our 2020 rankings.",{"entities": [(15, 19, "FAC")]}),
]

In [13]:
from __future__ import unicode_literals, print_function

import plac
import random
from pathlib import Path
import spacy
from spacy.util import minibatch, compounding


In [44]:
def main(model=None, output_dir=None, n_iter=100):
    """Load the model, set up the pipeline and train the entity recognizer."""
    if model is not None:
        nlp = spacy.load(model)  # load existing spaCy model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank("en")  # create blank Language class
        print("Created blank 'en' model")

    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe(ner, last=True)
    # otherwise, get it so we can add labels
    else:
        ner = nlp.get_pipe("ner")

    # add labels
    for _, annotations in TRAIN_DATA:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])

    # get names of other pipes to disable them during training
    pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]
    with nlp.disable_pipes(*other_pipes):  # only train NER
        # reset and initialize the weights randomly – but only if we're
        # training a new model
        if model is None:
            nlp.begin_training()
        for itn in range(n_iter):
            random.shuffle(TRAIN_DATA)
            losses = {}
            # batch up the examples using spaCy's minibatch
            # Original paramets 4,32.0,1.001 - drop=0.5
            batches = minibatch(TRAIN_DATA, size=compounding(4.0, 16.0, 1.001))
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(
                    texts,  # batch of texts
                    annotations,  # batch of annotations
                    drop=0.3,  # dropout - make it harder to memorise data
                    losses=losses,
                )
            print("Losses", losses)

    # test the trained model
    for text, _ in TRAIN_DATA:
        doc = nlp(text)
        print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
        print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])

    # save model to output directory
    if output_dir is not None:
        output_dir = Path(output_dir)
        if not output_dir.exists():
            output_dir.mkdir()
        nlp.to_disk(output_dir)
        print("Saved model to", output_dir)

        # test the saved model
        print("Loading from", output_dir)
        nlp2 = spacy.load(output_dir)
        for text, _ in TRAIN_DATA:
            doc = nlp2(text)
            print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
            print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])
            print('\n')


In [30]:
main(model="en_core_web_sm",output_dir='models')

Loaded model 'en_core_web_sm'
Losses {'ner': 12.883841872215271}
Losses {'ner': 6.783821161836386}
Losses {'ner': 6.568712592124939}
Losses {'ner': 8.046797275543213}
Losses {'ner': 7.733770430088043}
Losses {'ner': 13.5060555934906}
Losses {'ner': 6.725125730037689}
Losses {'ner': 5.707939859479666}
Losses {'ner': 8.729207932949066}
Losses {'ner': 8.428431697189808}
Losses {'ner': 11.105809725821018}
Losses {'ner': 7.687758911401033}
Losses {'ner': 7.325376391410828}
Losses {'ner': 6.685686994343996}
Losses {'ner': 8.967748216353357}
Losses {'ner': 12.601458430290222}
Losses {'ner': 8.173704486340284}
Losses {'ner': 4.524859876080882}
Losses {'ner': 6.511355206370354}
Losses {'ner': 4.723407041281462}
Losses {'ner': 5.914133947342634}
Losses {'ner': 4.4019126067869365}
Losses {'ner': 6.1727133728563786}
Losses {'ner': 3.482139964122325}
Losses {'ner': 4.448577044531703}
Losses {'ner': 6.702444531023502}
Losses {'ner': 8.710126876831055}
Losses {'ner': 3.3960008919239044}
Losses {'ner'

In [32]:
#Models
!ls models/*

models/meta.json  models/tokenizer

models/ner:
cfg  model  moves

models/parser:
cfg  model  moves

models/tagger:
cfg  model  tag_map

models/vocab:
key2row  lexemes.bin  lookups.bin  strings.json  vectors


In [33]:
#Check Model

nlp=spacy.load('models')

In [34]:
#Entities after train the model

doc = nlp(text)
displacy.render(doc, style="ent")

In [41]:
TRAIN_DATA = [
    ("all-electric trucks and SUVs as well as a self-driving vehicle", {"entities": [(24,27, "FAC")]}),
    ("unveiled by its subsidiary Cruise", {"entities": [(28, 34, "PRODUCT")]}),
    ("The Cruise Origin is powered by a brand-new",{"entities": [(4, 10, "PRODUCT")]}),
    ("Find out which SUVs are #1 in our 2020 rankings.",{"entities": [(15, 19, "FAC")]}),
    ("Searching for your next SUV?",{"entities": [(24, 27, "FAC")]}),
    ("View our list of the best SUVs for 2019 and 2020",{"entities": [(26, 30, "FAC")]}),
    ("We're Cruise, a self-driving car service designed for the cities we love.",{"entities": [(7, 13, "FAC")]})
]

In [45]:
main(model="en_core_web_sm",output_dir='models')

Loaded model 'en_core_web_sm'
Losses {'ner': 21.481184027332347}
Losses {'ner': 16.63173909705847}
Losses {'ner': 16.136904407292604}
Losses {'ner': 20.048798925243318}
Losses {'ner': 11.652754259426729}
Losses {'ner': 15.702799022663385}
Losses {'ner': 12.625489555521199}
Losses {'ner': 9.533809976652265}
Losses {'ner': 11.863633703440428}
Losses {'ner': 15.089138383278623}
Losses {'ner': 6.7890611588954926}
Losses {'ner': 4.730964160291478}
Losses {'ner': 9.653484001522884}
Losses {'ner': 6.8741117641329765}
Losses {'ner': 3.7424398034781916}
Losses {'ner': 6.094449054056895}
Losses {'ner': 6.3438728073379025}
Losses {'ner': 4.821898327070812}
Losses {'ner': 2.780504637138847}
Losses {'ner': 2.63661163062676}
Losses {'ner': 5.775917422834027}
Losses {'ner': 3.6542249140984495}
Losses {'ner': 3.170647745389033}
Losses {'ner': 9.097553916275501}
Losses {'ner': 3.2202579287756805}
Losses {'ner': 6.6548912623097625}
Losses {'ner': 7.362447573696372}
Losses {'ner': 0.3433494392837133}
Los

In [46]:
nlp=spacy.load('models')
doc = nlp(text)
displacy.render(doc, style="ent")